# Modulo appdata

## ParseTicker

In [1]:
class BaseParser():
    
    def __init__(self, source, target):
        self.__source = source
        self.__target = target
        self.__exceptions ={('Wiki','YahooFinance'):{'IBEX35': '^IBEX','CAC40': '^FCHI','DAX30': '^GDAXI', 'SP500':'^GSPC'},
                            ('Wiki', 'Ticker_Key'):{'IBEX35': '^IBEX.TT','CAC40': '^FCHI.TT','DAX30': '^GDAXI.TT', 'SP500':'^GSPC.TT'},
                            ('Wiki','AlphaVantage'):{}}
        
    def get_exceptions(self):
        return self.__exceptions
    
    def set_source(self, source):
        print("Set ticker source in parser")
        self.__source = source
    
    def get_source(self):
        return self.__source
    
    def set_target(self, target):
        print("Set ticker target in parser")
        self.__target = target
    
    def get_target(self):
        return self.__target
       
    def parse(self, name, prefix, suffix, excedent_string):
        key_tuple = (self.__source,self.__target) # Tuple key to select Source-Target exceptions
        exception_dict = self.__exceptions.get(key_tuple, {}) #FIX inform not properly loaded. Mirar longitud del dict.

        if name in exception_dict:
            return exception_dict.get(name, name) 
        
        name = name.replace(excedent_string,'')
        return prefix + name + suffix
    
    def parse_markets(self, raw_ticker, parser_keys): # Feeds tickers for Yahoo finance downloader based on market. Uses parse from BaseParser
        try:
            return self.parse(raw_ticker['ticker'], 
                     parser_keys[('market', raw_ticker['market'])]['prefix'],
                     parser_keys[('market', raw_ticker['market'])]['suffix'],
                     parser_keys[('market', raw_ticker['market'])]['excedent_string'])
        except:
            return raw_ticker['ticker']

In [2]:
class WikiToYahooFinance_Parser(BaseParser):
    
    def __init__(self):
        super().__init__(source='Wiki', target='YahooFinance')
        
        self.__parser_keys_feed = {('market','IBEX35'):{'prefix':'','suffix':'.MC','excedent_string':''},
                              ('market','CAC40'):{'prefix':'','suffix':'.PA','excedent_string':'Euronext: '},
                              ('market','DAX30'):{'prefix':'','suffix':'.DE', 'excedent_string':''},
                              ('market','SP500'):{'prefix':'','suffix':'', 'excedent_string':''}}
        
        
    def feeder_ticker(self,feed_ticker): # Appends feeds to original dictionary, under 'feeds' label. Uses parse_market.
        feed_ticker['feeds'] = {'name':self.get_target(),'ticker':self.parse_markets(feed_ticker, self.__parser_keys_feed)} #Feeds
        return feed_ticker

In [3]:
class WikiToTickerKey_Parser(BaseParser):
    
    def __init__(self):
        super().__init__(source='Wiki', target='YahooFinance')
        
        self.__parser_keys_tickerKey = {('market','IBEX35'):{'prefix':'','suffix':'.MC.TT','excedent_string':''},
                                        ('market','CAC40') :{'prefix':'','suffix':'.PA.TT','excedent_string':'Euronext: '},
                                        ('market','DAX30') :{'prefix':'','suffix':'.DE.TT', 'excedent_string':''},
                                        ('market','SP500') :{'prefix':'','suffix':'', 'excedent_string':''}}
        
    def feeder_ticker(self,feed_ticker): # Appends feeds to original dictionary, under 'feeds' label. Uses parse_market.
        feed_ticker['tickerKey'] = self.parse_markets(feed_ticker, self.__parser_keys_tickerKey)
        return feed_ticker


In [4]:
## Housekeeping module

In [5]:
import os
from os import path 
import json

class data_housekeeper():
    
    def __init__(self):
        pass

        
    @staticmethod
    def create_dir(myDir):
        try:
            if not path.exists(myDir):
                print('Creating dir: ', myDir)
                os.mkdir(myDir)
        except AccessDeniedError as err:
            print('No dir created: ', err)

    @staticmethod
    def delete_dir(myDir):
        try:
            if path.exists(myDir):
                print('Deleting dir: ', myDir)
                os.rmdir(myDir)
            else:
                print('Inexistent folder')
        except AccessDeniedError as err:
            print('No dir deleted: ', err)

    @staticmethod
    def delete_non_empty_dir(myDir):
        try:
            if path.exists(myDir):
                print('Deleting dir: ', myDir)
                shutil.rmtree(myDir)
        except:
            print('No dir deleted: ')        

    @staticmethod
    def create_nested_dir_path(dir_list):
        nested_path = '/' + '/'.join(dir_list)
        return nested_path
    
    
    @staticmethod
    def create_file_path_in_nested_dir(dir_list, file_name):
        nested_dir_path_ = data_housekeeper.create_nested_dir_path(dir_list)
        print('nested_dir_path: ', nested_dir_path_)
        file_path_in_nested_dir_ = nested_dir_path_ + '/' + file_name
        return file_path_in_nested_dir_
    

    @staticmethod
    def create_nested_dir(dir_path):
        try:
            myPath = ''
            for directory in dir_path:
                myPath += './' + directory
                data_housekeeper.create_dir(myPath)
            #print(myPath)
        except:
            print('No nested dir created')
            #print(myPath)

    @staticmethod
    def create_nested_dir_in_parent_dir(dir_path , n):
        try:
            origin_wd = os.getcwd()
            for n in range(n):
                path_parent = os.path.dirname(os.getcwd())
                os.chdir(path_parent)
            root_dir = os.getcwd()
            print('Root dir: ', root_dir)
            data_housekeeper.create_nested_dir(dir_path)
            os.chdir(origin_wd)
            return root_dir
        except:
            os.chdir(origin_wd)
            print('Error: No directory created. Posible causes: \nWrong type imput arguments \nUnable to access to nested dir')
            return os.getcwds()
                
    @staticmethod
    def list_dict_to_json(dir_list,upper_stages,file_name, dictionary_list):
        parent_dir_path = data_housekeeper.create_nested_dir_in_parent_dir(dir_list,upper_stages)
        print('parent_dir_path: ', parent_dir_path)
        print('dir_list: ', dir_list)
        file_path = parent_dir_path + data_housekeeper.create_file_path_in_nested_dir(dir_list, file_name)
        print('file path: ', file_path)
        with open(file_path, 'w') as json_file:
            json.dump(dictionary_list, json_file, indent=4)

    @staticmethod
    def load_json_to_list(dir_list, file_name):
        relative_file_path_ = '.' + data_housekeeper.create_file_path_in_nested_dir(dir_list, file_name)
        print(relative_file_path_)
        with open(relative_file_path_) as json_file:
            data = json.load(json_file)
        return data

In [6]:
## TickerManager module

In [7]:
from abc import ABC, abstractmethod
from datetime import datetime

class TickerManager(ABC):

    def __init__(self, tickermanager_id, tickers_list):
        #Variables comunes a todos los tipos de tickermanager.
        self.__tickermanager_id = tickermanager_id
        self.__tickers_list = tickers_list # Lista de diccionarios. Ej:
        # [{ "ticker":"BBVA(raw)", "market":"IBEX35", "hora":"YYYY/MM/DD/HH/MM/SS", "source":"Wikipedia", "tickerYahoofinance":"BBVA.MC", "tickerAlphaVantage":"BBVA.BM(inventado)"}]
        self.__update_date = [] #FIX, eliminar, esta variable está incluida en el diccionario
        self.__tickers_config = []
    
    @abstractmethod
    def get_raw_tickers(self):
        pass

    @abstractmethod
    def get_parsed_tickers(self):
        pass
    
    @abstractmethod
    def get_save_tickers(self):
        pass

    def set_tickers_list(self, tickers_list):
        # Sobreescribe la lista tickers_list
        print("Set tickers_list")
        self.__tickers_list = tickers_list
    
    def get_tickers_list(self):
        # Devuelve la lista tickers_list
        print('Getting tickers_list')
        return self.__tickers_list
    
    def set_tickers_config(self, tickers_config):
        # Sobreescribe la lista tickers_config
        print("Set tickers_config")
        self.__tickers_config = tickers_config
    
    def get_tickers_config(self):
        # Devuelve la lista tickers_config
        print('Getting tickers_config')
        return self.__tickers_config
    
    def get_num_tickers(self):
        # Devuelve el número de diccionarios(uno por cada ticker) en la tickers_list
        print("Number of tickers: {}".format(len(self.__tickers_list)))
        return len(self.__tickers_list)

    def set_tickermanager_id(self, tickermanager_id):
        # Actualiza el par-value del diccionario: source. Ej. "source":"Wikipedia"
        print("Set tickermanager_id: {}".format(tickermanager_id))
        self.__tickermanager_id = tickermanager_id
    
    def get_tickermanager_id(self):
        print("TickerManager id is: {}".format(self.__tickermanager_id))
        return self.__tickermanager_id
    
    def set_update_date(self, update_date):
        # Actualiza el par-value del diccionario: "hora":"YYYY/MM/DD/HH/MM/SS"
        print("Set date of update: {}".format(update_date))
        self.__update_date = update_date #FIX. Actualiza diccionario, no nueva variable

    def get_update_date(self):
        # Devuelve el par-value del diccionario: "hora":"YYYY/MM/DD/HH/MM/SS"
        print("Last date of update id is: {}".format(self.__update_date))
        return self.__update_date
    
    def save_tickers_list(self):
        # Guarda la ticker list. Guarda en JSON.
        dir_list = ['Data']
        #dir_list = True
        upper_stages = 0
        file_name = 'tickers_list.json'
        data_housekeeper.list_dict_to_json(dir_list,upper_stages,file_name,self.get_tickers_list())
        print("Save tickers list")
    
    def save_tickers_config(self):
        now = datetime.now()
        ticker_list_metadata_ = [{"timestamp":now.strftime("%d/%m/%Y %H:%M:%S")}]
        tickers_config_ = {'metadata':ticker_list_metadata_, 'data':self.get_tickers_list()}
        self.set_tickers_config(tickers_config_)
        # Config parameters to save
        dir_list = ['Data']
        upper_stages = 0
        file_name = 'tickers_config.json'
        data_housekeeper.list_dict_to_json(dir_list,upper_stages,file_name,self.get_tickers_config())
        
    

In [10]:
import re
import os
import bs4 as bs
import pandas as pd
import requests
from datetime import date
from datetime import datetime


class TickerManager_Wiki(TickerManager):

    def __init__(self):
        super().__init__(tickermanager_id = "Wiki", tickers_list=[])
        #Variables que sólo sean características del tipo WIKI. Market, url y posiciones en la tabla wiki
        #FIX 
        self.__wiki_metadata = [{'market':'IBEX35','url':'https://es.wikipedia.org/wiki/IBEX_35','pos_table':{'ticker':0, 'company':1,'sector':4,'entry_date':3,'ISIN':5}},
                {'market':'DAX30','url':'https://de.wikipedia.org/wiki/DAX','pos_table':{'ticker':1, 'company':0,'sector':2,'entry_date':5}},
                {'market':'CAC40','url':'https://es.wikipedia.org/wiki/CAC_40','pos_table':{'ticker':2,'company':0, 'sector':1}},
                {'market':'SP500','url':'http://en.wikipedia.org/wiki/List_of_S%26P_500_companies','pos_table':{'ticker':0, 'company':1,'sector':3,'sub_industry':4,'entry_date':6,'CIK':7}}
                ]
        self.__paths = []
        # Instanciar aquí el parser. Para este tickermanger, tener una lista de parsers.
        self.__my_parser_W2YF = WikiToYahooFinance_Parser()
        self.__my_parser_W2TK = WikiToTickerKey_Parser()
        # El código puede ser genérico, 
    
    #Métodos que sólo son característicos del tipo Wiki
    def get_wikitable_from_url(self,wiki_metadata) -> dict:
        today = date.today()
        resp = requests.get(wiki_metadata['url'])
        soup = bs.BeautifulSoup(resp.text, 'lxml')
        table = soup.find('table', {'class': 'wikitable sortable'})
        wikitable_data = []
        for row in table.findAll('tr')[1:]:
            wikitable_dict = {}
            for key in wiki_metadata['pos_table']:
                data = row.findAll('td')[wiki_metadata['pos_table'][key]].text
                wikitable_dict[key] = data.strip()
            wikitable_dict['market'] = wiki_metadata['market']
            wikitable_dict['active_type'] = 'stock'
            wikitable_dict['active_flag'] = True
            wikitable_dict['timestamp'] = today.strftime("%d-%m-%Y")
            wikitable_dict['data_update'] = '-'
            wikitable_data.append(wikitable_dict)
        wikitable_data.append({'ticker': wiki_metadata['market'] , 
                               'market': wiki_metadata['market'], 
                               'timestamp':today.strftime("%d-%m-%Y"),
                               'data_update': '-', 
                               'active_type':'ETF'})
        return wikitable_data
    
    def get_raw_tickers(self):
        # Actualiza los par-value del diccionario: Ticker:value, market:value, name:value.
        # Descargar
        raw_ticker_list = []
        print('Start raw ticker data extraction from Wikipedia ')
        for i,metadata in enumerate(self.__wiki_metadata):
            print('Getting data from market:{}, site:{}'.format(metadata['market'], metadata['url']))
            #print(i), print(metadata)
            wikitable_data = self.get_wikitable_from_url(metadata)
            raw_ticker_list += wikitable_data
        print('Extraction completed')
        super().set_tickers_list(raw_ticker_list)


    #def get_parsed_tickers(self):
    #    # Actualiza los par-value del diccionario: "tickerYahoofinance":"value", "tickerAlphaVantage":"value"
    #    print('Activation: get_parsed_tickers')
    
    def get_parsed_tickers(self): # Función con la que hay que engancahr
        '''
        # Actualiza los par-value del diccionario: "tickerYahoofinance":"value", "tickerAlphaVantage":"value"
        print('Activation: get_parsed_tickers')
        feeder_ticker_list = self.__my_parser_W2YF.parse_all(self.get_tickers_list())
        self.set_tickers_list(feeder_ticker_list)
        #return feeder_ticker_list
        '''
        # Actualiza los par-value del diccionario: "tickerYahoofinance":"value", "tickerAlphaVantage":"value"
        print('Activation: get_parsed_tickers')
        raw_ticker_list = self.get_tickers_list()
        feeder_ticker_list = []
        for raw_ticker in raw_ticker_list:
            feeder_ticker = self.__my_parser_W2TK.feeder_ticker(raw_ticker)
            feeder_ticker = self.__my_parser_W2YF.feeder_ticker(feeder_ticker)
            feeder_ticker_list.append(feeder_ticker)
            
        print('All tickers fed')
        self.set_tickers_list(feeder_ticker_list)
        #return feeder_ticker_list
        
    def Myfun(self):
        #Necesito una función que descargue, parsee y guarde tickers en variable interna
        pass
        
    def get_save_tickers_market(self,market):
        print('Activation: get_save_tickers_market. Getting tickers with source in Wikipedia. Market: {}'.format(market))
        pass
        # Función alto nivel de la clase TickerManager_Wiki. 
        # Obtiene una lista de diccionarios, un diccionario por ticker con origen Wikipedia para el mercado especificado y lo guarda.
    
    def get_save_tickers(self):
        print('Get tickers with source in Wikipedia. All markets')
        pass
        # Función alto nivel de la clase TickerManager_Wiki. 
        # Obtiene una lista de diccionarios, un diccionario por ticker con origen Wiki, para cada uno de los mercados que existen y lo guarda.
        # Incluye los siguientes métodos
        
    def compare_tickers_config(self, tickers_new, tickers_config):
        tickerKey_list_config = [d['tickerKey'] for d in tickers_config]
        tickerKey_list_new = [d['tickerKey'] for d in tickers_new]
        timestamp_new_values =  tickers_new[0]['timestamp']

        # Loop in tickers_config. Update coincident tickers
        for ticker_config in tickers_config:
            ticker_config['active_flag'] = False # By default false 
            if ticker_config.get('active_type') == 'stock': # Tickers of stock type. 
                if ticker_config['tickerKey'] in tickerKey_list_new: #In case appear in new vales. Update Timestamp and active flag
                    ticker_config['timestamp'] = timestamp_new_values # deepcopy
                    ticker_config['active_flag'] = True # deepcopy
            if ticker_config.get('active_type') == 'ETF':
                ticker_config['active_flag'] = True # Siempre true, no pueden desaparecer. deepcopy

        # Loop in new_tickers. Update new tickers tickers of stock type. Add full ticker.
        for ticker in tickers_new:
            if ticker['tickerKey'] not in tickerKey_list_config:
                tickers_config.append(ticker)
        return tickers_config
    
    def update_tickers_config(self):
        #Initialize names of variables
        dir_list_tickers_config = ['Data','Tickers','Dummy1']
        upper_stages_tickers_config = 0
        file_name_tickers_config = 'tickers_config.json'
        file_path_tickers_config =  '.' + data_housekeeper.create_file_path_in_nested_dir(dir_list_tickers_config, file_name_tickers_config)

        # Download new set of tickers and parse it.
        self.get_raw_tickers()
        self.get_parsed_tickers()
        new_tickers = self.get_tickers_list()
        #data = tickers_list

        # Load previous tickers_config. If not present, then create from scratch
        if not path.isfile(file_path_tickers_config):
            print('No file tickers_config.json detected, creating..')
            #data_housekeeper.list_dict_to_json(dir_list_tickers_config,upper_stages_tickers_config,file_name_tickers_config,new_tickers)
            self.save_tickers_config()
            print('Created tickers_config.json')
            return
        tickers_config = data_housekeeper.load_json_to_list(dir_list_tickers_config, file_name_tickers_config)
        tickers_config = tickers_config['data']

        # Compare new tickers with tickers_config
        tickers_config = self.compare_tickers_config(new_tickers, tickers_config)
        self.set_tickers_config(tickers_config)
        # Save updated tickers config
        self.save_tickers_config()


In [11]:
tickerMan = TickerManager_Wiki()
tickerMan.update_tickers_config()

nested_dir_path:  /Data/Tickers/Dummy1
Start raw ticker data extraction from Wikipedia 
Getting data from market:IBEX35, site:https://es.wikipedia.org/wiki/IBEX_35
Getting data from market:DAX30, site:https://de.wikipedia.org/wiki/DAX
Getting data from market:CAC40, site:https://es.wikipedia.org/wiki/CAC_40
Getting data from market:SP500, site:http://en.wikipedia.org/wiki/List_of_S%26P_500_companies
Extraction completed
Set tickers_list
Activation: get_parsed_tickers
Getting tickers_list
All tickers fed
Set tickers_list
Getting tickers_list
nested_dir_path:  /Data/Tickers/Dummy1
./Data/Tickers/Dummy1/tickers_config.json
Set tickers_config
Getting tickers_list
Set tickers_config
Getting tickers_config
Root dir:  C:\Users\alvaro\Repos\Python_knowledge\Appdatabase project
parent_dir_path:  C:\Users\alvaro\Repos\Python_knowledge\Appdatabase project
dir_list:  ['Data']
nested_dir_path:  /Data
file path:  C:\Users\alvaro\Repos\Python_knowledge\Appdatabase project/Data/tickers_config.json


## Testing outputs

In [ ]:
tickerMan = TickerManager_Wiki()
tickerMan.get_tickermanager_id()
tickerMan.set_update_date('1981')
tickerMan.get_update_date()
tickerMan.set_tickermanager_id('Wiki2')
tickerMan.get_tickermanager_id()
tickerMan.get_num_tickers()
tickerMan.get_num_tickers()
tickerMan.get_save_tickers_market('IBEX35')
tickerMan.get_save_tickers()
tickerMan.get_raw_tickers()
tickerMan.get_parsed_tickers()

In [ ]:
tickerMan.get_tickers_list()

In [ ]:
tickerMan.save_tickers_list()

In [ ]:
tickerMan.save_tickers_config()

In [ ]:
tickerMan.update_tickers_config()

## Development layer

In [ ]:
tickerMan = TickerManager_Wiki()
tickerMan.get_parsed_tickers()

In [ ]:
tickerMan.get_tickers_list()

In [ ]:
if not tickerMan.get_tickers_list():
    print('empty')

In [ ]:
list1 = [{"Alvaro":"1.72"},
         {"Ricardo": "1,79"},
         {"Ferri":"1,7"}
        ]
len(list1)

In [ ]:
class TickerManagerFactory:
    def __init__(self) -> object:
        pass
    
    @staticmethod
    def get_names() -> list:
        tickermanager_names = []
        tickermanager_names.append('TickerManager_Wiki')
        return tickermanager_names
    
    @staticmethod
    def create(tickermanager_name, *args) -> TickerManager :
        if tickermanager_name == 'TickerManager_Wiki':
            return TickerManager_Wiki()

        else:
            # TODO raise exception
            print('Create TickerManager: Unknown Tickermanager {}'.format(tickermanager_name))
            return None
    
    
    

In [ ]:
tickerMan2 = TickerManagerFactory.create('TickerManager_Wiki')

In [ ]:
tickerMan2 = TickerManager_Wiki()
tickerMan2.get_tickermanager_id()
tickerMan2.set_update_date('1981')
tickerMan2.get_update_date()
tickerMan2.set_tickermanager_id('Wiki2')
tickerMan2.get_tickermanager_id()
tickerMan2.get_raw_tickers()
tickerMan2.get_num_tickers()
tickerMan2.get_num_tickers()
tickerMan2.get_parsed_tickers()
tickerMan2.save_tickers_list()
tickerMan2.get_save_tickers_market('IBEX35')
tickerMan2.get_save_tickers()